In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('C:/Users/Nurtilek/Downloads/salary_train.csv')
pred = pd.read_csv('C:/Users/Nurtilek/Downloads/salary_predict.csv')
df_sub = pd.read_csv('C:/Users/Nurtilek/Downloads/salary_example_submition.csv')

In [3]:
pred_data = pred.drop(['salary'], axis=1)
pred_data

,Id,algebra,programming,data science,robotics,economics,job
0,9000,73,59,57,54,61,robotics engineer
1,9001,77,80,53,93,80,senior developer
2,9002,95,72,88,63,84,developer
3,9003,83,88,97,75,50,robotics engineer
4,9004,52,85,92,87,62,junior developer
...,...,...,...,...,...,...,...
995,9995,83,98,71,83,61,junior developer
996,9996,98,59,74,79,52,economist
997,9997,91,68,76,50,92,developer
998,9998,61,70,95,51,87,junior developer


In [4]:
df['job'].unique()

array(['junior developer', 'data scientist', 'developer', 'economist',
       'robotics engineer', 'senior developer'], dtype=object)

In [5]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
df['job']= label_encoder.fit_transform(df['job']) 
pred_data['job']= label_encoder.fit_transform(pred_data['job']) 

In [6]:
print(label_encoder.classes_)

['data scientist' 'developer' 'economist' 'junior developer'
 'robotics engineer' 'senior developer']


In [7]:
df.head()

,Id,algebra,programming,data science,robotics,economics,job,salary
0,0,87,62,86,61,90,3,140000
1,1,76,84,76,80,79,0,780000
2,2,56,55,99,82,98,1,210000
3,3,99,66,65,84,58,2,420000
4,4,73,87,56,84,73,0,760000


In [8]:
df_y = df['salary']
df_x = df.drop(['salary'], axis=1)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_x)
scaler.fit(pred_data)

StandardScaler()

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3)

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [12]:
lin_reg = LinearRegression(n_jobs=-1)
lin_reg.fit(x_train, y_train)
reg_predictions = lin_reg.predict(x_test)

tree = DecisionTreeRegressor()
tree.fit(x_train, y_train)
tree_predictions = tree.predict(x_test)

forest = RandomForestRegressor()
forest.fit(x_train, y_train)
forest_predictions = forest.predict(x_test)

In [13]:
print(mean_squared_error(y_test, reg_predictions, squared=False))

272177.3725171917


In [14]:
print(mean_squared_error(y_test, forest_predictions, squared=False))

50755.37898879582


In [15]:
print(mean_squared_error(y_test, tree_predictions, squared=False))

71404.4297827004


In [16]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [17]:
rf_random = RandomizedSearchCV(estimator = forest, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.4min finished


{'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 100,
 'bootstrap': True}

In [18]:
forest = RandomForestRegressor(n_estimators = 1000, max_depth=100, min_samples_split=5, min_samples_leaf=4,
                               bootstrap= True, max_features='auto', )
forest.fit(x_train, y_train)
forest_predictions = forest.predict(x_test)

In [19]:
print(mean_squared_error(y_test, forest_predictions, squared=False))

50238.465998989224


In [20]:
predictions = forest.predict(pred_data)

In [23]:
df_sub['salary'] = predictions.astype(int)
df_sub.to_csv('Nurti_5.csv', index=False)